In [133]:
import pandas as pd
import seaborn as sns

In [143]:
from collections import Counter

def proc_file(filer, printer=False):
    times = Counter()
    open_times = {}

    stk = []
    stk_times = [[0,0]]

    dff = []

    with open("/users/danielwong/Downloads/"+filer) as f:
        for line in f:
            if "[python]" in line:
                line = line.split()
                if line[1].startswith('==='):
                    idx = line[1][3:][:-1]
                    stk.append(idx)
                    open_times[idx] = int(line[2]) / 1000.
                    bt_times = Counter()
                    stk_times.append([0,0])
    #         print(idx)
                else:
                    idx = line[1][:-4]
                    tx = int(line[2]) / 1000.
                    assert idx == stk.pop(), (idx, stk.pop(), line)
                    elapsed_inner, kernel = stk_times.pop()
    #         del open_times[idx]
    #         print(tx)
                    elapsed = tx - open_times[idx]
                    times[idx] += elapsed
    #         print(bt_times)
                    kernel = sum(bt_times.values()) or kernel
                    stk_times[-1][0] += elapsed
                    stk_times[-1][1] += kernel
                    if printer:
                            print("{}{} kernel={:g} ({:.1f}) inner={:g} ({:.1f}) elapsed={:g}".format(" "*len(stk),idx, kernel, kernel/elapsed*100, elapsed_inner, elapsed_inner/elapsed*100, elapsed))
                    dff.append({'idx': idx, 'kernel': kernel, 'elapsed': elapsed, 'inner': elapsed_inner})
                    bt_times = Counter()

    #         print(idx, tx)
            elif ", start" in line:
                line = line.split()
                idx = line[0]
                tx = float(line[4].replace(":", "."))
                open_times[idx] = tx
            elif ", end" in line:
                line = line.split()
                idx = line[0]
                tx = float(line[4].replace(":", ".").replace(",",""))
                try:
                    bt_times[idx] += (tx - open_times[idx] + 100) % 100
                    xx = tx - open_times[idx]
                    del open_times[idx]
                except KeyError:
                    print "Bad line", line
    #       if xx < 0:
    #         print xx, tx, open_times[idx]
    times
    #open_times
    return dff

# proc_file("output/cifar10/190907_profile/nofilter_cifar10_wideresnet_0_128_1024_3_trial1_seed1337_v5")

In [147]:
files = """./output/cifar10/190907_profile/sb-relative-alwayson_cifar10_wideresnet_0_128_1024_3_trial1_seed1337_v5
./output/cifar100/190907_profile/nofilter_cifar100_wideresnet_0_128_1024_3_trial1_seed1337_v5
./output/cifar100/190907_profile/sb-relative-alwayson_cifar100_wideresnet_0_128_1024_3_trial1_seed1337_v5""".split("\n")

for fn in files:
    dff = proc_file(fn)
    df = pd.DataFrame(dff)
    print(fn)
    print(len(dff))
    df_ = df[ df['elapsed'] < 1.2]
    print "Outliers:", len(df[ df['elapsed'] > 1.2])
    print(df_.groupby('idx')['elapsed'].mean())
    print(df_.groupby('idx')['elapsed'].count())
#     print(df_.groupby('idx')['elapsed'].describe())

./output/cifar10/190907_profile/sb-relative-alwayson_cifar10_wideresnet_0_128_1024_3_trial1_seed1337_v5
3520
Outliers: 9
idx
backward      0.334084
epoch         0.424323
forward       0.161813
forward-sb    0.150395
Name: elapsed, dtype: float64
idx
backward       584
epoch         1169
forward        587
forward-sb    1171
Name: elapsed, dtype: int64
./output/cifar100/190907_profile/nofilter_cifar100_wideresnet_0_128_1024_3_trial1_seed1337_v5
3519
Outliers: 5
idx
backward    0.335700
epoch       0.515753
forward     0.162749
Name: elapsed, dtype: float64
idx
backward    1171
epoch       1171
forward     1172
Name: elapsed, dtype: int64
./output/cifar100/190907_profile/sb-relative-alwayson_cifar100_wideresnet_0_128_1024_3_trial1_seed1337_v5
3518
Outliers: 6
idx
backward      0.334021
epoch         0.423178
forward       0.160724
forward-sb    0.150113
Name: elapsed, dtype: float64
idx
backward       584
epoch         1170
forward        586
forward-sb    1172
Name: elapsed, dtype: int

In [ ]:
# xx = df.groupby('idx')['elapsed'].sum()
# xx, (xx['forward']+xx['backward']+xx['forward-sb'])/xx['epoch']